In [150]:
#Note - when using covariates, we need to introduce a model matrix.
#Here it appears directly before the ComBat function is applied.

In [151]:
import pandas as pd
import numpy.linalg as la
import numpy as np
import patsy
import sys

In [152]:
def design_mat(mod, numerical_covariates, batch_levels):
    # require levels to make sure they are in the same order as we use in the
    # rest of the script.
    design = patsy.dmatrix("~ 0 + C(batch, levels=%s)" % str(batch_levels),
                                                  mod, return_type="dataframe")

    mod = mod.drop(["batch"], axis=1)
    numerical_covariates = list(numerical_covariates)
    sys.stderr.write("found %i batches\n" % design.shape[1])
    other_cols = [c for i, c in enumerate(mod.columns)
                  if not i in numerical_covariates]
    factor_matrix = mod[other_cols]
    design = pd.concat((design, factor_matrix), axis=1)
    if numerical_covariates is not None:
        sys.stderr.write("found %i numerical covariates...\n"
                            % len(numerical_covariates))
        for i, nC in enumerate(numerical_covariates):
            cname = mod.columns[nC]
            sys.stderr.write("\t{0}\n".format(cname))
            design[cname] = mod[mod.columns[nC]]
    sys.stderr.write("found %i categorical variables:" % len(other_cols))
    sys.stderr.write("\t" + ", ".join(other_cols) + '\n')
    return design

In [153]:
def aprior(gamma_hat):
    m = gamma_hat.mean()
    s2 = gamma_hat.var()
    return (2 * s2 +m**2) / np.float64(s2)

In [154]:
def bprior(gamma_hat):
    m = gamma_hat.mean()
    s2 = gamma_hat.var()
    return (m*s2+m**3) / np.float64(s2)

In [155]:
#Used to calculate gamma_star - appears in it_sol and it_sol_loc functions.
def postmean(g_hat, g_bar, n, d_star, t2):
    return (t2*n*g_hat+d_star * g_bar) / (t2*n+d_star)

In [156]:
def postvar(sum2, n, a, b):
    return (0.5 * sum2 + b) / (n / 2.0 + a - 1.0)

In [157]:
def it_sol(sdat, g_hat, d_hat, g_bar, t2, a, b, conv=0.0001):
    n = (1 - np.isnan(sdat)).sum(axis=1)
    g_old = g_hat.copy()
    d_old = d_hat.copy()

    change = 1
    count = 0
    while change > conv:
        #print g_hat.shape, g_bar.shape, t2.shape
        g_new = postmean(g_hat, g_bar, n, d_old, t2)
        sum2 = ((sdat - np.dot(g_new.reshape((g_new.shape[0], 1)), np.ones((1, sdat.shape[1])))) ** 2).sum(axis=1)
        d_new = postvar(sum2, n, a, b)
       
        change = max((abs(g_new - g_old) / g_old).max(), (abs(d_new - d_old) / d_old).max())
        g_old = g_new #.copy()
        d_old = d_new #.copy()
        count = count + 1
    adjust = (g_new, d_new)
    return adjust

In [158]:
def it_sol_loc(g_hat, g_bar, n, d_hat, t2):
    n = 1
    g_old = g_hat.copy()
    d_old = d_hat.copy()

    g_new = postmean(g_hat, g_bar, n, d_old, t2)
    g_old = g_new #.copy()
    adjust = (g_new)
    return adjust

In [159]:
def it_sol_scale(sdat, g_hat, d_hat, g_bar, t2, a, b, conv=0.0001):
    n = (1 - np.isnan(sdat)).sum(axis=1)
    g_old = g_hat.copy()
    d_old = d_hat.copy()

    change = 1
    count = 0
    while change > conv:
        #print g_hat.shape, g_bar.shape, t2.shape
        g_new = postmean(g_hat, g_bar, n, d_old, t2)
        sum2 = ((sdat - np.dot(g_new.reshape((g_new.shape[0], 1)), np.ones((1, sdat.shape[1])))) ** 2).sum(axis=1)
        d_new = postvar(sum2, n, a, b)
       
        change = (abs(d_new - d_old) / d_old).max()
        g_old = g_new #.copy()
        d_old = d_new #.copy()
        count = count + 1
    adjust = (d_new, g_new)
    return adjust

In [160]:
def combat(data, batch, model=None, numerical_covariates=None, mean_only=False, ref_batch=None):
    """Correct for batch effects in a dataset
    Parameters
    ----------
    data : pandas.DataFrame
        A (n_features, n_samples) dataframe of the expression to batch correct
    batch : List-like
        A column corresponding to the batches in the data, in the same order
        as the samples in ``data``
    model : patsy.design_info.DesignMatrix, optional
        A model matrix describing metadata on the samples which could be
        causing batch effects. If not provided, then will attempt to coarsely
        correct just from the information provided in ``batch``
    numerical_covariates : list-like
        List of covariates in the model which are numerical, rather than
        categorical
    mean_only : (T/F)
        Only adjusts for mean across the batches, forgoing scale (variance) adjustment. 
        Default = False
    ref_batch : int
        Adjusts all batches to the specified referance batch. Specified batch will not be adjusted.
        Default = None
        
    Returns
    -------
    corrected : pandas.DataFrame
        A (n_features, n_samples) dataframe of the batch-corrected data
    """
    if isinstance(numerical_covariates, str):
        numerical_covariates = [numerical_covariates]
    
    if numerical_covariates is None:
        numerical_covariates = []

    if model is not None and isinstance(model, pd.DataFrame):
        model["batch"] = list(batch)
    
    else:
        model = pd.DataFrame({'batch': batch})
        
    if ref_batch is not None and isinstance(ref_batch, int):
        ref = int(ref_batch)

    batch_items = model.groupby("batch").groups.items()
    batch_levels = [k for k, v in batch_items]
    batch_info = [v for k, v in batch_items]
    n_batch = len(batch_info)
    n_batches = np.array([len(v) for v in batch_info])
    n_array = sum(n_batches)
    
    drop_cols = [cname for cname, inter in  ((model == 1).all()).iteritems() if inter == True]
    drop_idxs = [list(model.columns).index(cdrop) for cdrop in drop_cols]
    model = model[[c for c in model.columns if not c in drop_cols]]
    numerical_covariates = [list(model.columns).index(c) if isinstance(c, str) else c
            for c in numerical_covariates if not c in drop_cols]
    
    design = design_mat(model, numerical_covariates, batch_levels)
    
    if ref_batch is not None:
        design.iloc[:,ref-1] = 1.
    
    sys.stderr.write("Standardizing Data across genes.\n")
    B_hat = np.dot(np.dot(la.inv(np.dot(design.T, design)), design.T), data.T)
    
    if ref_batch is not None:
        grand_mean = B_hat[ref-1]
        #Here, refdat cannot yet be defined because dataframe used for 'dft' does not have the 
        #actual genetic information. Thus, we first rename it refid (to minimize changes to
        #initial code). See below.
        refid = dft.loc[dft['batch'] == ref].drop('batch', axis=1)
        reflist = refid.index.tolist()
        #For bladderbatch data only, since initial set up for our 'dft' and bladder batch 'pheno'
        #is a little different, we have to adjust code. Namely, in our 'dft', I can directly pull 
        #out the genes by batch number, but with 'pheno' dataframe used in bladderbatch data we 
        #cannot, so we have to associate designated batch back with the bladderbatch data set.        
        refdat = dat[reflist].T
        #Now back to code as normal. This way, 'refdat' is now correct size and shape to be used in
        #original code, and we do not have to rename it.
        desbatch = design.T[reflist].T
        var_pooled = np.dot(((refdat - np.dot(desbatch, B_hat))**2).T, np.ones((n_batches[ref-1], 1)) / n_batches[ref-1])
    
    else:
        grand_mean = np.dot((n_batches / n_array).T, B_hat[:n_batch,:])
        var_pooled = np.dot(((data - np.dot(design, B_hat).T)**2), np.ones((n_array, 1)) / n_array)

    stand_mean = np.dot(grand_mean.T.reshape((len(grand_mean), 1)), np.ones((1, n_array)))
    tmp = np.array(design.copy())
    tmp[:,:n_batch] = 0
    stand_mean  += np.dot(tmp, B_hat).T
    
    s_data = ((data - stand_mean) / np.dot(np.sqrt(var_pooled), np.ones((1, n_array))))

    sys.stderr.write("Fitting L/S model and finding priors\n")
    batch_design = design[design.columns[:n_batch]]
    gamma_hat = np.dot(np.dot(la.inv(np.dot(batch_design.T, batch_design)), batch_design.T), s_data.T)

    delta_hat = []

    for i, batch_idxs in enumerate(batch_info):
        if mean_only is False:
            delta_hat.append(s_data[batch_idxs].var(axis=1))
        
        else:
            x=s_data[batch_idxs].var(axis=1)
            x[x<100]=1
            np.float64(delta_hat.append(x))
            
    gamma_bar = gamma_hat.mean(axis=1) 
    t2 = gamma_hat.var(axis=1)
    
    a_prior = list(map(aprior, delta_hat))
    b_prior = list(map(bprior, delta_hat))
    
    sys.stderr.write("Finding parametric adjustments\n")
    gamma_star, delta_star = [], []
    for i, batch_idxs in enumerate(batch_info):
    
        if mean_only is False:
            
            temp = it_sol(s_data[batch_idxs], gamma_hat[i],
            delta_hat[i], gamma_bar[i], t2[i], a_prior[i], b_prior[i])

            gamma_star.append(temp[0])
            delta_star.append(temp[1])
        
        else:
            
            temploc = it_sol_loc(gamma_hat[i], gamma_bar[i], 1, delta_hat[i], t2[i])

            gamma_star.append(temploc)
        
            tempscale = it_sol_scale(s_data[batch_idxs], gamma_hat[i],
                     delta_hat[i], gamma_bar[i], t2[i], a_prior[i], b_prior[i])
        
            y=tempscale[1]
            y[y<100]=1
            delta_star.append(y)
    
    sys.stdout.write("Adjusting data\n")
    bayesdata = s_data
    gamma_star = np.array(gamma_star)
    delta_star = np.array(delta_star)
    
    if ref_batch is not None:
        gamma_star[ref-1]=0
        delta_star[ref-1]=1

    for j, batch_idxs in enumerate(batch_info):

        dsq = np.sqrt(delta_star[j,:])
        dsq = dsq.reshape((len(dsq), 1))
        denom =  np.dot(dsq, np.ones((1, n_batches[j])))
        numer = np.array(bayesdata[batch_idxs] - np.dot(batch_design.ix[batch_idxs], gamma_star).T)

        bayesdata[batch_idxs] = numer / denom
   
    vpsq = np.sqrt(var_pooled).reshape((len(var_pooled), 1))
    bayesdata = bayesdata * np.dot(vpsq, np.ones((1, n_array))) + stand_mean
   
    return bayesdata

In [161]:
#Import data.
dft = pd.read_table('../Data/Python_Data/bladder-pheno.txt', index_col=0)
dat = pd.read_table('../Data/Python_Data/bladder-expr.txt', index_col=0)

In [162]:
#Let's use age and cancer status as covariates.
dft.head()

,sample,outcome,batch,cancer,age
cel,,,,,
GSM71019.CEL,1,Normal,3,Normal,1
GSM71020.CEL,2,Normal,2,Normal,2
GSM71021.CEL,3,Normal,2,Normal,3
GSM71022.CEL,4,Normal,3,Normal,4
GSM71023.CEL,5,Normal,3,Normal,5


In [163]:
#To do so, we must construct a model matrix from the pheno array to use as the model argument 
#in ComBat. As shown below, cancer status (whether Normal, Cancer, or Biopsy) has been converted to 
#boolean.
mod = patsy.dmatrix("~age+cancer", dft, return_type = "dataframe")
mod.head()

,Intercept,cancer[T.Cancer],cancer[T.Normal],age
cel,,,,
GSM71019.CEL,1.0,0.0,1.0,1.0
GSM71020.CEL,1.0,0.0,1.0,2.0
GSM71021.CEL,1.0,0.0,1.0,3.0
GSM71022.CEL,1.0,0.0,1.0,4.0
GSM71023.CEL,1.0,0.0,1.0,5.0


In [164]:
#Run ComBat using model matrix highlighting age and cancer status as covariates, with age 
#specified as numerical covariate, and adjusting only for location (mean) - not for
#scale (variance). Additionally, we are adjusting everything to batch number 2 - which itself will
#not change.
cmdf = combat(dat, dft.batch, mod, "age", True, 2)

found 5 batches
found 1 numerical covariates...
	age
found 2 categorical variables:	cancer[T.Cancer], cancer[T.Normal]
Standardizing Data across genes.
Fitting L/S model and finding priors
C:\Users\Moime\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  after removing the cwd from sys.path.
Finding parametric adjustments
C:\Users\Moime\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # This is added back by InteractiveShellApp.init_path()


Adjusting data


C:\Users\Moime\Anaconda3\lib\site-packages\ipykernel_launcher.py:153: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


In [165]:
cmdf.head()

,GSM71019.CEL,GSM71020.CEL,GSM71021.CEL,GSM71022.CEL,GSM71023.CEL,GSM71024.CEL,GSM71025.CEL,GSM71026.CEL,GSM71028.CEL,GSM71029.CEL,...,GSM71068.CEL,GSM71069.CEL,GSM71070.CEL,GSM71071.CEL,GSM71072.CEL,GSM71073.CEL,GSM71074.CEL,GSM71075.CEL,GSM71076.CEL,GSM71077.CEL
1007_s_at,9.297060,8.628044,8.779235,8.430459,9.438731,9.205023,9.108034,8.735616,10.036121,10.168602,...,10.815742,9.735264,9.639089,9.329659,9.244777,8.628912,9.136315,9.227774,9.267002,8.765379
1053_at,5.293953,5.063598,5.113116,5.128195,5.130168,5.197203,5.252312,5.220931,5.520059,5.025180,...,5.540214,5.057161,5.142740,5.017566,5.277536,5.138645,5.175671,5.129212,5.097005,5.141493
117_at,6.783043,6.663625,6.465892,6.551440,6.415475,6.231173,6.414849,6.846798,6.087455,6.352257,...,6.159465,6.429851,6.486004,6.640066,6.671011,6.618497,6.341321,6.337025,6.532518,6.839865
121_at,9.379328,9.439977,9.540738,9.731956,9.275675,8.480459,9.093704,9.263386,8.099955,9.834564,...,8.360712,8.817219,9.395608,9.128504,9.176797,9.015099,9.686451,8.990390,9.257966,10.073165
1255_g_at,4.177761,4.466027,4.144885,4.399427,4.288598,4.129828,4.402590,4.173666,3.699066,4.338196,...,3.883611,4.033592,4.360601,4.249672,4.106061,4.327778,4.563188,4.226531,4.322274,4.915744
